In [4]:
import duckdb

# Connect to (or create) a DuckDB file
con = duckdb.connect("crashes.duckdb")

# Create a table inside DuckDB from the Parquet file
con.execute("""
    CREATE OR REPLACE TABLE crashes AS 
    SELECT 
        BOROUGH,
        EXTRACT(YEAR FROM "CRASH DATE") AS YEAR,
        COALESCE("NUMBER OF PERSONS KILLED", 0) AS PERSONS_KILLED,
        COALESCE("NUMBER OF PERSONS INJURED", 0) AS PERSONS_INJURED
    FROM 'crashes.parquet'
""")

In [5]:
print(con.execute("SELECT COUNT(*) FROM crashes").fetchall())

[(1989065,)]


In [6]:
# create a persistent connection to the DuckDB file
con = duckdb.connect("crashes.duckdb")

In [12]:
df = con.execute("SELECT * FROM crashes").df()
df.head()

,BOROUGH,YEAR,PERSONS_KILLED,PERSONS_INJURED
0,BROOKLYN,2023,0.0,1.0
1,BROOKLYN,2021,0.0,0.0
2,BROOKLYN,2021,0.0,0.0
3,BROOKLYN,2021,0.0,0.0
4,BRONX,2021,0.0,2.0


In [13]:
df.shape

(1989065, 4)